In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("metr-evals/daft-math")


dataset.csv:   0%|          | 0.00/185k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/199 [00:00<?, ? examples/s]

In [9]:
columns= ds.column_names["train"]
print(columns)

['Original Question', 'Integer Answer Variant Question', 'Estimated Difficulty', 'Original Answer', 'Integer Variant Answer', 'Source']


In [14]:
import json,re,os,unicodedata
field_map={
    "problem": "Integer Answer Variant Question",
    "gt_answer": "Integer Variant Answer",
    "level": "",
    "type": ""
}

def normalize_answer(s: str):
    s = unicodedata.normalize("NFKC", s).strip()
    # strip LaTeX \boxed{} if present
    boxed = re.findall(r"\\boxed\\{([^}]*)\\}", s)
    if boxed:
        return boxed[-1].strip()
    return s

def convert(save_dir="converted_daft_math"):
    ds = load_dataset("metr-evals/daft-math")
    os.makedirs(save_dir, exist_ok=True)
    for split, split_ds in ds.items():
        out_path = os.path.join(save_dir, f"{split}.jsonl")
        with open(out_path, "w", encoding="utf-8") as f:
            for i, row in enumerate(split_ds):
                item = {
                    "problem_idx": f"{split}_{i}",
                    "problem": row[field_map["problem"]],
                    "gt_answer": normalize_answer(str(row[field_map["gt_answer"]])),
                    "level": row.get("level", ""),   # leave blank if absent
                    "type": row.get("topic", row.get("type", "")),
                    "source": "daft-math",
                    "split": split
                }
                if field_map.get("solution") in row and row[field_map["solution"]]:
                    item["solution"] = row[field_map["solution"]]
                item["correct_answers"] = [item["gt_answer"]]
                f.write(json.dumps(item, ensure_ascii=False) + "\n")
        print("Wrote", out_path)

if __name__ == "__main__":
    convert()


Wrote converted_daft_math/train.jsonl
